# Limpieza de los datos

In [58]:
import pandas as pd
import pickle

In [59]:
# aquí están los datos en local
#data_path = '/Users/cbautistap/Dropbox/MCD/Cursos/primavera-2021/dpa/data/Food_Inspections.csv'

In [60]:
# cargamos datos, convertimos a formato datetime la columna 'Inspection Date' y queda con formato yy/mm/dd
#data = pd.read_csv(data_path, parse_dates=['Inspection Date'])

In [61]:
data = pd.DataFrame(pickle.load(open('../temp/historic-inspections-2021-03-15.pkl', 'rb')))

In [62]:
data.shape

(217281, 17)

In [63]:
data.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2492768,MARZ COMMUNITY BREWING CO.,MARZ COMMUNITY BREWING CO.,2776267,Restaurant,Risk 2 (Medium),1950 N WESTERN AVE,CHICAGO,IL,60647,2021-03-12T00:00:00.000,License,Pass,41.917134801370274,-87.68754385201615,"{'latitude': '-87.68754385201615', 'longitude'...",NaN
1,2492746,LAKE & PULASKI CHILD DEVELOPMENT CENTER,LAKE & PULASKI CHILD DEVELOPMENT CENTER,2216861,Daycare Above and Under 2 Years,Risk 1 (High),316 N PULASKI RD,CHICAGO,IL,60624,2021-03-11T00:00:00.000,License,Pass w/ Conditions,41.885919645895974,-87.72596362714377,"{'latitude': '-87.72596362714377', 'longitude'...","3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
2,2492736,"BROTHER'S SUBMARINE & CHICKEN, INC.",BROTHER'S SUBMARINE & CHICKEN,2464826,Restaurant,Risk 1 (High),1200 W 63RD ST,CHICAGO,IL,60636,2021-03-11T00:00:00.000,Canvass,Pass w/ Conditions,41.77971768255646,-87.65456552453057,"{'latitude': '-87.65456552453057', 'longitude'...","1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW..."
3,2492763,PERA TURKISH KITCHEN AND BAR,PERA TURKISH KITCHEN AND BAR,2129482,Restaurant,Risk 1 (High),2833 N BROADWAY,CHICAGO,IL,60657,2021-03-11T00:00:00.000,Canvass,Out of Business,41.93368774378592,-87.64445413171818,"{'latitude': '-87.64445413171818', 'longitude'...",NaN
4,2492737,ANTICO,ANTICO,2031732,Restaurant,Risk 1 (High),1946 N LEAVITT ST,CHICAGO,IL,60647,2021-03-11T00:00:00.000,Non-Inspection,No Entry,41.91755731772555,-87.68266439539492,"{'latitude': '-87.68266439539492', 'longitude'...",NaN


In [64]:
#data['month'] = data['Inspection Date'].dt.month

Ya no es necesario pues al descargar datos desde la api los nombres de las columnas ya están limpios

In [65]:
# creamos función para estandarizar el nombre de las columnas.
#def standarize_column_names(data, excluded_punctuation=".,-*¿?¡!#"):
#    data.columns = data.columns.str.lower().str.replace(" ", "_")
#    for ch in excluded_punctuation:
#        data.columns = data.columns.str.replace(ch, "")

In [66]:
#standarize_column_names(data)

In [67]:
# cambiamos el nombre de la columna license
data=data.rename(columns={'license_':'license'})
data.head(2)

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2492768,MARZ COMMUNITY BREWING CO.,MARZ COMMUNITY BREWING CO.,2776267,Restaurant,Risk 2 (Medium),1950 N WESTERN AVE,CHICAGO,IL,60647,2021-03-12T00:00:00.000,License,Pass,41.917134801370274,-87.68754385201615,"{'latitude': '-87.68754385201615', 'longitude'...",NaN
1,2492746,LAKE & PULASKI CHILD DEVELOPMENT CENTER,LAKE & PULASKI CHILD DEVELOPMENT CENTER,2216861,Daycare Above and Under 2 Years,Risk 1 (High),316 N PULASKI RD,CHICAGO,IL,60624,2021-03-11T00:00:00.000,License,Pass w/ Conditions,41.885919645895974,-87.72596362714377,"{'latitude': '-87.72596362714377', 'longitude'...","3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."


In [68]:
data.dtypes

inspection_id      object
dba_name           object
aka_name           object
license            object
facility_type      object
risk               object
address            object
city               object
state              object
zip                object
inspection_date    object
inspection_type    object
results            object
latitude           object
longitude          object
location           object
violations         object
dtype: object

Queremos convertir license y zip a string. Si lo convertimos directo astype(str) queda con decimales, por lo tanto convertimos primero a formato int. Sin embargo, ambas columnas tienen NAs por lo que una solución es llenarlos con 0 y ahora sí convertir a string. QUIZÁ NO HAY PROBLEMA TENERLOS EN STRING CON EL DECIMAL XXXX.0

In [69]:
data.isna().sum()

inspection_id          0
dba_name               0
aka_name            2492
license               17
facility_type       4900
risk                  69
address                0
city                 163
state                 50
zip                   47
inspection_date        0
inspection_type        1
results                0
latitude             731
longitude            731
location             731
violations         58061
dtype: int64

In [70]:
#Subset de data frame con missing values en cualquier columna
#null_data = data[data.isnull().any(axis=1)]
#null_data.head()

### Queremos entender qué tipo de observaciones son las que tienen missing values, con el objetivo de encontrar algún patrón o algo que nos permita imputar de mejor manera. 

Para eso, haremos subset de missing values por columna. Empezamos con las que tienen mayor NaN.

Nota. La columna de aka_name no la exploraremos. Es lógico que haya mv para esta columna, además, no la utilizaremos para el modelo

In [71]:
#queremos ver que observaciones son las que tienen missing values en la columna 'violations'
#null_violations = null_data[null_data.violations.isnull()]
#null_violations.shape

In [72]:
#Missing values en la columna 'violations'
#null_data[null_data.violations.isnull()].groupby('results').count()

Agrupando por Results, el 89.4% de los NaN de la columna 'Violations' están en 'Pass', 'Out of Business' y 'No Entry'. Tiene sentido que los que pasaron la inspección no tengan violaciones, asimismo, que los que ya estén cerrados y en los cuales no se entró, no tengan violaciones.

Por otro lado, solo el 5% de los mv de la columna 'Violations' son de resultados 'Fail' (que representan 1.47% del total de observaciones).
Dentro de éstas, el 62% son de Tipo de Inspección 'License'

In [73]:
#results_fail = null_violations['results']=='Fail'

In [74]:
#null_violations[results_fail].groupby('inspection_type').count()

In [75]:
# MV de columna 'Facility Type'
#null_data[null_data.facility_type.isnull()].groupby('results').count()

El 84% de los NaN en la columna de Facility Type corresponden a results 'Out of business', 'Business Not Located' y 'No Entry'

In [76]:
# MV de columna 'location'
#null_data[null_data.location.isnull()].groupby('results').count()

En total son 710 observaciones con mv en ubicación (que coinciden con coordenadas geográficas). La mayoría son de establecimientos que pasaron la visita. Asimismo, destaca que por facility type, la mayoría son de tipo school y también incluye algunos establecimientos móviles.

In [77]:
# MV de columna 'location', agrupando con columna 'facility_type'
#null_data[null_data.location.isnull()].groupby('facility_type').count()

In [78]:
# MV de columna 'risk', agrupando por 'results'
#null_data[null_data.risk.isnull()].groupby('results').count()

In [79]:
# MV de columna 'risk', agrupando por 'inspection_type'
#null_data[null_data.risk.isnull()].groupby('inspection_type').count()

## Por lo pronto, para proseguir con limpieza de datos, se decide reemplazar los MV con 0 para variables numéricas y con el valor 'na' para variables de texto.

### En primer lugar, convertimos a variables string las de license y zip 

In [80]:
# Reemplazamos con 0 los mv en estas dos columnas
data['license']=data['license'].fillna(0)
data['zip']=data['zip'].fillna(0)

In [81]:
data.isna().sum()

inspection_id          0
dba_name               0
aka_name            2492
license                0
facility_type       4900
risk                  69
address                0
city                 163
state                 50
zip                    0
inspection_date        0
inspection_type        1
results                0
latitude             731
longitude            731
location             731
violations         58061
dtype: int64

In [82]:
# convertimos columnas 'license' y 'zip'. Primero a int y luego a str
data = data.astype({"license": 'int', "zip": 'int'})
data = data.astype({"license": 'str', "zip": 'str'})
data.dtypes

inspection_id      object
dba_name           object
aka_name           object
license            object
facility_type      object
risk               object
address            object
city               object
state              object
zip                object
inspection_date    object
inspection_type    object
results            object
latitude           object
longitude          object
location           object
violations         object
dtype: object

In [85]:
data.zip.dtype

dtype('O')

In [36]:
data.head(2)

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2492768,MARZ COMMUNITY BREWING CO.,MARZ COMMUNITY BREWING CO.,2776267,Restaurant,Risk 2 (Medium),1950 N WESTERN AVE,CHICAGO,IL,60647,2021-03-12T00:00:00.000,License,Pass,41.917134801370274,-87.68754385201615,"{'latitude': '-87.68754385201615', 'longitude'...",NaN
1,2492746,LAKE & PULASKI CHILD DEVELOPMENT CENTER,LAKE & PULASKI CHILD DEVELOPMENT CENTER,2216861,Daycare Above and Under 2 Years,Risk 1 (High),316 N PULASKI RD,CHICAGO,IL,60624,2021-03-11T00:00:00.000,License,Pass w/ Conditions,41.885919645895974,-87.72596362714377,"{'latitude': '-87.72596362714377', 'longitude'...","3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."


In [37]:
# Hacemos subset de las columnas que utilizaremos para el modelo
data = data[['inspection_id', 'facility_type', 'risk', 'zip', 'inspection_date','inspection_type', 'results', 'violations', 'latitude', 'longitude']]
data.head()

,inspection_id,facility_type,risk,zip,inspection_date,inspection_type,results,violations,latitude,longitude
0,2492768,Restaurant,Risk 2 (Medium),60647,2021-03-12T00:00:00.000,License,Pass,NaN,41.917134801370274,-87.68754385201615
1,2492746,Daycare Above and Under 2 Years,Risk 1 (High),60624,2021-03-11T00:00:00.000,License,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.885919645895974,-87.72596362714377
2,2492736,Restaurant,Risk 1 (High),60636,2021-03-11T00:00:00.000,Canvass,Pass w/ Conditions,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.77971768255646,-87.65456552453057
3,2492763,Restaurant,Risk 1 (High),60657,2021-03-11T00:00:00.000,Canvass,Out of Business,NaN,41.93368774378592,-87.64445413171818
4,2492737,Restaurant,Risk 1 (High),60647,2021-03-11T00:00:00.000,Non-Inspection,No Entry,NaN,41.91755731772555,-87.68266439539492


In [38]:
data.isnull().sum()

inspection_id          0
facility_type       4900
risk                  69
zip                    0
inspection_date        0
inspection_type        1
results                0
violations         58061
latitude             731
longitude            731
dtype: int64

In [39]:
data[['facility_type', 'risk', 'inspection_type','violations']]=data[['facility_type', 'risk', 'inspection_type','violations']].fillna('na')

In [40]:
data.isnull().sum()

inspection_id        0
facility_type        0
risk                 0
zip                  0
inspection_date      0
inspection_type      0
results              0
violations           0
latitude           731
longitude          731
dtype: int64

In [41]:
# Cambiamos valor de results a 'Pass' y 'Not Pass' para cualquier otro
data['results'].mask(data['results'] != 'Pass', other = 'Not Pass', inplace=True)

In [42]:
data.groupby('results').count()

,inspection_id,facility_type,risk,zip,inspection_date,inspection_type,violations,latitude,longitude
results,,,,,,,,,
Not Pass,103704,103704,103704,103704,103704,103704,103704,103422,103422
Pass,113577,113577,113577,113577,113577,113577,113577,113128,113128


Ahora, estandarizamos los valores de las columnas (quitamos acentos y eliminamos caracteres especiales)

In [43]:
def standarize_column_strings(data, columns, excluded_punctuation=".,-*'¿?¡!()", gap_punct="\/", special_chr="|"):
    for col in columns:
        data[col] = data[col].apply(lambda x: x.lower().replace(" ", "_"))
        data[col] = data[col].apply(lambda x: x.lower().replace("á", "a"))
        data[col] = data[col].apply(lambda x: x.lower().replace("é", "e"))
        data[col] = data[col].apply(lambda x: x.lower().replace("í", "i"))
        data[col] = data[col].apply(lambda x: x.lower().replace("ó", "o"))
        data[col] = data[col].apply(lambda x: x.lower().replace("ú", "u"))
        data[col] = data[col].apply(lambda x: x.lower().replace("ü", "u"))
        for ch in excluded_punctuation:
            data[col] = data[col].apply(lambda x: x.replace(ch, ""))
        for ch in gap_punct:
            data[col] = data[col].apply(lambda x: x.replace(ch,"_"))
        for ch in special_chr:
            data['violations'] = data['violations'].apply(lambda x: x.replace(ch,"~"))

In [33]:
#def standarize_column_strings(data, columns, excluded_punctuation=".,-*¿?¡!"):
#    for col in columns:
#        data[col] = data[col].apply(lambda x: x.lower().replace(" ", "_"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("á", "a"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("é", "e"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("í", "i"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("ó", "o"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("ú", "u"))
#        data[col] = data[col].apply(lambda x: x.lower().replace("ü", "u"))
#        for ch in excluded_punctuation:
#            data[col] = data[col].apply(lambda x: x.replace(ch, ""))

In [44]:
#Convertimos variables de texto a formato estándar
col_text = data.select_dtypes(include=['object']).columns.to_list()
standarize_column_strings(data, col_text)
data.head()

AttributeError: 'float' object has no attribute 'lower'

In [45]:
# Función para reemplazar el valor de una columna específica
def replace_column_value(data, column, search_string, replace_string):
    data[column] = data[column].apply(lambda x: x.replace(search_string, replace_string))

In [46]:
replace_column_value(data,'facility_type', 'restuarant', 'restaurant')
replace_column_value(data,'facility_type', 'theatre', 'theater')
replace_column_value(data,'facility_type', 'herabal', 'herbal')
replace_column_value(data,'facility_type', 'day_care', 'daycare')
replace_column_value(data,'facility_type', 'long_term', 'longterm')


#'theatre', 'theater')) %>% 
#  mutate_at(vars(facility_type), ~str_replace_all(., 'herabal', 'herbal')) %>% 
#  mutate_at(vars(facility_type), ~str_replace_all(., 'day care', 'daycare')) %>% 
#  mutate_at(vars(facility_type), ~str_replace_all(., 'long term', 'longterm'))

In [47]:
data.loc[data['facility_type'].str.contains('childern|children|1023|5_years_old', case=False, na=None), 'facility_type'] = 'childrens_service_facility'
data.loc[data['facility_type'].str.contains('conv|mart|gas_station_store', case=False, na=None), 'facility_type'] = 'convenience_store'
data.loc[data['facility_type'].str.contains('assis|longterm|nursing|supportive', case=False, na=None), 'facility_type'] = 'assisted_living'
data.loc[data['facility_type'].str.contains('herbal_life|herbalife|herbalcal', case=False, na=None), 'facility_type'] = 'herbalife'
data.loc[data['facility_type'].str.contains('after_school', case=False, na=None), 'facility_type'] = 'after_school'
data.loc[data['facility_type'].str.contains('tavern|pub|brew|wine_tasting|bar_grill|hooka', case=False, na=None), 'facility_type'] = 'bar'
data.loc[data['facility_type'].str.contains('bakery', case=False, na=None), 'facility_type'] = 'bakery'
data.loc[data['facility_type'].str.contains('mobil|truck|mfd', case=False, na=None), 'facility_type'] = 'mobile_food'
data.loc[data['facility_type'].str.contains('kitchen', case=False, na=None), 'facility_type'] = 'kitchen'
data.loc[data['facility_type'].str.contains('restaurant|rstaurant|diner', case=False, na=None), 'facility_type'] = 'restaurant'
data.loc[data['facility_type'].str.contains('retail', case=False, na=None), 'facility_type'] = 'retail'
data.loc[data['facility_type'].str.contains('roof', case=False, na=None), 'facility_type'] = 'rooftop'
data.loc[data['facility_type'].str.contains('grocery', case=False, na=None), 'facility_type'] = 'grocery_store'
data.loc[data['facility_type'].str.contains('liquor', case=False, na=None), 'facility_type'] = 'liquor'
data.loc[data['facility_type'].str.contains('popup', case=False, na=None), 'facility_type'] = 'popup_establishment'
data.loc[data['facility_type'].str.contains('school|college|shcool', case=False, na=None), 'facility_type'] = 'school'
data.loc[data['facility_type'].str.contains('daycare', case=False, na=None), 'facility_type'] = 'daycare'
data.loc[data['facility_type'].str.contains('cafeteria|coffee|cafe', case=False, na=None), 'facility_type'] = 'coffee'
data.loc[data['facility_type'].str.contains('drug_store|pharmacy', case=False, na=None), 'facility_type'] = 'drug_store'
data.loc[data['facility_type'].str.contains('gym|fitness|weight_loss|exercise', case=False, na=None), 'facility_type'] = 'gym'
data.loc[data['facility_type'].str.contains('commissary|machine|commiasary', case=False, na=None), 'facility_type'] = 'vending_machine'
data.loc[data['facility_type'].str.contains('ice_cream|paleteria|gelato', case=False, na=None), 'facility_type'] = 'ice_cream'
data.loc[data['facility_type'].str.contains('banquet', case=False, na=None), 'facility_type'] = 'banquet'
data.loc[data['facility_type'].str.contains('lounge', case=False, na=None), 'facility_type'] = 'lounge'
data.loc[data['facility_type'].str.contains('church|religious', case=False, na=None), 'facility_type'] = 'church'
data.loc[data['facility_type'].str.contains('kiosk', case=False, na=None), 'facility_type'] = 'kiosk'
data.loc[data['facility_type'].str.contains('health|rehab', case=False, na=None), 'facility_type'] = 'health'
data.loc[data['facility_type'].str.contains('event', case=False, na=None), 'facility_type'] = 'events'
data.loc[data['facility_type'].str.contains('donut|hotdog|hot_dog|popcorn|juice|tea|dessert|deli|salad|snack|candy|shake|watermelon|smoothie|food|sushi', case=False, na=None), 'facility_type'] = 'other_food'
data.loc[data['facility_type'].str.contains('poultry|butcher|slaughter|meat', case=False, na=None), 'facility_type'] = 'butcher'
data.loc[data['facility_type'].str.contains('profit', case=False, na=None), 'facility_type'] = 'non_profit'
data.loc[data['facility_type'].str.contains('na', case=False, na=None), 'facility_type'] = 'not_specified'



Se reducen a 121 diferentes categorías, probablemente se puedan agrupar más

In [48]:
data.nunique()

inspection_id      217281
facility_type         121
risk                    5
zip                   113
inspection_date      2833
inspection_type        98
results                 2
violations         158096
latitude            17291
longitude           17291
dtype: int64

In [49]:
data['facility_type'].unique()

array(['restaurant', 'daycare', 'school', 'bakery',
       'childrens_service_facility', 'grocery_store', 'not_specified',
       'liquor', 'hair_salon', 'butcher', 'bar', 'kitchen',
       'assisted_living', 'coffee', 'catering', 'wholesale', 'hospital',
       'other_food', 'vending_machine', 'banquet', 'ice_cream',
       'popup_establishment', 'hotel', 'gas_station', 'convenience_store',
       'shelter', 'non_profit', 'greenhouse', 'herbalife', 'health',
       'events', 'private_club', 'gym', 'art_center', 'rooftop',
       'dining_hall', 'general_store', 'repackaging_plant', 'retail',
       'store', 'drug_store', 'blockbuster_video', 'service_gas_station',
       'unused_storage', 'distribution_center', 'movie_theater', 'kiosk',
       'theater', 'hostel', 'charter', 'herbal_drinks', 'lounge',
       'dollar_tree', 'church', 'other', 'youth_housing', 'stadium',
       'french_market_space', 'regulated_business', 'dollar_store',
       'tobacco_store', 'beverage_silverware_wareh

In [50]:
# Ahora hacemos limpieza similar para columna de 'inspection_type'
data.loc[data['inspection_type'].str.contains('license', case=False, na=None), 'inspection_type'] = 'license'
data.loc[data['inspection_type'].str.contains('task_force|taskforce', case=False, na=None), 'inspection_type'] = 'task_force'
data.loc[data['inspection_type'].str.contains('canvass|canvas', case=False, na=None), 'inspection_type'] = 'canvas'
data.loc[data['inspection_type'].str.contains('complaint', case=False, na=None), 'inspection_type'] = 'complaint'
data.loc[data['inspection_type'].str.contains('food|sick', case=False, na=None), 'inspection_type'] = 'suspected_food_poisoning'

Se reducen a 45 categorías

In [51]:
data.nunique()

inspection_id      217281
facility_type         121
risk                    5
zip                   113
inspection_date      2833
inspection_type        45
results                 2
violations         158096
latitude            17291
longitude           17291
dtype: int64

In [52]:
data['inspection_type'].unique()

array(['license', 'canvas', 'noninspection', 'complaint',
       'recent_inspection', 'suspected_food_poisoning', 'consultation',
       'office_assignment', 'not_ready', 'special_events_festivals',
       'fire', 'tag_removal', 'na', 'kitchen_closed_for_renovation', 'ob',
       'corrective_action', 'task_force', 'illegal_operation',
       'package_liquor_1474', 'out_of_business', 'no_entry',
       'sample_collection', 'fire_complain', 'addendum', 'sfp',
       'expansion', 'sfp_recently_inspected', 'tavern_1470', 'citf',
       'kids_cafe', 'error_save', 'changed_court_date', 'liquor_catering',
       'summer_feeding', 'citation_reissued', 'taste_of_chicago',
       'haccp_questionaire', 'out_ofbusiness', 'duplicated',
       'recall_inspection', 'reinspection_of_48_hour_notice',
       'reinspection', 'business_not_located', 'reinspection_of_closeup',
       'possible_fbi'], dtype=object)

Ahora con columna 'violations'

In [54]:
#data['violations'].unique()

Por último, queremos contar el número de violaciones

In [55]:
data['num_violations'] = data['violations'].apply(lambda x: x.count('_~_') + 1 if x != 'na' else 0)

In [56]:
data.groupby('num_violations').count()

,inspection_id,facility_type,risk,zip,inspection_date,inspection_type,results,violations,latitude,longitude
num_violations,,,,,,,,,,
0,58061,58061,58061,58061,58061,58061,58061,58061,57865,57865
1,19464,19464,19464,19464,19464,19464,19464,19464,19370,19370
2,25515,25515,25515,25515,25515,25515,25515,25515,25385,25385
3,26798,26798,26798,26798,26798,26798,26798,26798,26702,26702
4,23504,23504,23504,23504,23504,23504,23504,23504,23432,23432
5,18708,18708,18708,18708,18708,18708,18708,18708,18650,18650
6,13833,13833,13833,13833,13833,13833,13833,13833,13808,13808
7,9606,9606,9606,9606,9606,9606,9606,9606,9587,9587
8,6814,6814,6814,6814,6814,6814,6814,6814,6795,6795


In [49]:
data.to_csv('/Users/cbautistap/Dropbox/MCD/Cursos/primavera-2021/dpa/data/food_inspections_clean.csv')

In [48]:
data.head(10000).to_csv('/Users/cbautistap/Dropbox/MCD/Cursos/primavera-2021/dpa/data/sample_cbp.csv')